# Opérateurs logiques MongoDB : `$and`, `$or`, `$nor`, `$not` 

Ce notebook présente les opérateurs logiques de MongoDB à travers des exemples simples et accessibles.  
L'objectif est de montrer comment filtrer des documents à l'aide de conditions logiques dans une base MongoDB.

Nous allons :
- Créer une connexion à MongoDB local (via Docker)
- Insérer des données fictives
- Illustrer les opérateurs logiques un par un

In [6]:
# Connexion à MongoDB local (via Docker)
from pymongo import MongoClient
import pandas as pd

# Connexion à l'instance MongoDB (sur localhost:27017)
client = MongoClient("mongodb://localhost:27017/")

# Connexion à la base testdb
db = client.testdb

# Sélection de la collection "clients"
collection = db.clients

In [7]:
# On supprime les documents existants pour éviter les doublons
collection.delete_many({})

# Insertion de plusieurs documents (clients fictifs)
collection.insert_many([
    { "name": "Alice", "age": 22, "city": "Lyon" },
    { "name": "César", "age": 31, "city": "Lille" },
    { "name": "Thomas", "age": 24, "city": "Nantes" },
    { "name": "Fréderic", "age": 35, "city": "Paris" },
    { "name": "Jawad", "age": 36, "city": "Paris" }
])

InsertManyResult([ObjectId('6888a4f7ceaa36f271fac3f3'), ObjectId('6888a4f7ceaa36f271fac3f4'), ObjectId('6888a4f7ceaa36f271fac3f5'), ObjectId('6888a4f7ceaa36f271fac3f6'), ObjectId('6888a4f7ceaa36f271fac3f7')], acknowledged=True)

In [8]:
# Affichage des clients insérés dans la collection
docs = list(collection.find())
pd.DataFrame(docs)

,_id,name,age,city
0,6888a4f7ceaa36f271fac3f3,Alice,22,Lyon
1,6888a4f7ceaa36f271fac3f4,César,31,Lille
2,6888a4f7ceaa36f271fac3f5,Thomas,24,Nantes
3,6888a4f7ceaa36f271fac3f6,Fréderic,35,Paris
4,6888a4f7ceaa36f271fac3f7,Jawad,36,Paris


## Opérateur logique `$and`

L’opérateur `$and` permet de filtrer les documents qui respectent toutes les conditions spécifiées.  
C’est l’équivalent du **ET logique**.

---

**Exemple simple :**  
On veut retrouver les personnes :
- qui ont 30 ans ou plus
- **et** qui habitent à Paris

Cela revient à dire :  
> "Je veux les personnes qui ont 30 ans ou plus **ET** qui sont à Paris."


In [9]:
# Requête : âge ≥ 30 ET ville = Paris
query = {
    "$and": [
        { "age": { "$gte": 30 } },
        { "city": "Paris" }
    ]
}

# Exécution de la requête
results = list(collection.find(query))

# Affichage du résultat dans un tableau
pd.DataFrame(results)


,_id,name,age,city
0,6888a4f7ceaa36f271fac3f6,Fréderic,35,Paris
1,6888a4f7ceaa36f271fac3f7,Jawad,36,Paris


## Opérateur logique `$or`

L'opérateur `$or` permet de sélectionner les documents qui remplissent au moins une des conditions spécifiées.  
Il fonctionne comme un **OU logique**.

---

**Exemple pédagogique :**  
Je veux les personnes :
- qui ont moins de 25 ans
- **ou** qui habitent à Lyon

Cela veut dire :  
> "Si l’une des deux conditions est vraie, la personne est sélectionnée."

In [10]:
# Requête : âge < 25 OU ville = Lyon
query = {
    "$or": [
        { "age": { "$lt": 25 } },
        { "city": "Lyon" }
    ]
}

# Exécution de la requête
results = list(collection.find(query))

# Affichage
pd.DataFrame(results)

,_id,name,age,city
0,6888a4f7ceaa36f271fac3f3,Alice,22,Lyon
1,6888a4f7ceaa36f271fac3f5,Thomas,24,Nantes


## Opérateur logique `$nor`

L’opérateur `$nor` permet de rejeter tous les documents qui remplissent l’une ou l’autre des conditions.  
C’est l’équivalent de dire : "**ni... ni...**".

---

**Exemple pédagogique :**  
Je veux exclure les personnes :
- qui ont plus de 30 ans
- **ou** qui habitent à Paris

Cela revient à dire :  
> "Je veux uniquement les personnes **qui n’ont pas plus de 30 ans** **et** **qui n’habitent pas à Paris**."

In [11]:
# Requête : exclure les personnes ayant age > 30 ou habitant à Paris
query = {
    "$nor": [
        { "age": { "$gt": 30 } },
        { "city": "Paris" }
    ]
}

# Exécution de la requête
results = list(collection.find(query))

# Affichage
pd.DataFrame(results)

,_id,name,age,city
0,6888a4f7ceaa36f271fac3f3,Alice,22,Lyon
1,6888a4f7ceaa36f271fac3f5,Thomas,24,Nantes


## Opérateur logique `$not`

L’opérateur `$not` permet de nier une seule condition spécifique.  
C’est une négation, mais elle s’utilise uniquement à **l’intérieur d’une condition**.

---

**Exemple pédagogique :**  
Je veux les personnes dont l’âge n’est pas supérieur ou égal à 30 ans.

Cela revient à dire :  
> "Je veux ceux qui **ont moins de 30 ans**."

À ne pas confondre avec `$nor`, qui agit sur plusieurs conditions.


In [12]:
# Requête : sélectionner les personnes dont l'âge n'est pas supérieur ou égal à 30
query = {
    "age": {
        "$not": { "$gte": 30 }
    }
}

# Exécution
results = list(collection.find(query))

# Affichage
pd.DataFrame(results)


,_id,name,age,city
0,6888a4f7ceaa36f271fac3f3,Alice,22,Lyon
1,6888a4f7ceaa36f271fac3f5,Thomas,24,Nantes


---

## Conclusion

Dans ce notebook, nous avons exploré les **opérateurs logiques** de MongoDB :  
- `$and` : pour combiner plusieurs conditions (ET)
- `$or` : pour accepter au moins une condition (OU)
- `$nor` : pour exclure toutes les conditions (NI... NI...)
- `$not` : pour nier une seule condition spécifique

Chaque opérateur a été présenté avec un exemple simple et compréhensible, en se basant sur une collection de clients fictifs.

Ce travail permet de mieux comprendre comment MongoDB peut filtrer des données avec souplesse, même sans être développeur.

---

**Prochaine étape :**  
Passer au commit Git pour enregistrer le travail dans la branche `feature/jawad`.
